





# The Iterated Shrinking Pie Tournament Ruleset
#### version 0:1

# Table of Contents
1. [Abstract](#abstract)
2. [The Rules](#rules)
3. [API & Running the game](#api)
4. [Future Research](#future-research)



## Abstract <a name="abstract"></a>

>The shrinking pie game, derived from the Rubinstein bargaining model, has two players negotiate how to split a fixed number of points between them (Rubinstein 1982). Each turn they do not come to an agreement, the ‘pie’ (points available to be split) shrinks according to a discount parameter. This project presents a ruleset that expands the two-player game to a tournament with three or more players negotiating in pairs. Notably, players have a new action available: the option to leave a negotiation for the prospect of striking a deal with a more amenable player. This may present a better model for real-world bargaining environments, particularly where players have multiple potential partners. Additionaly, this project offers software which implements the ruleset, developed for easy creation and testing of strategies and bargaining environments. The software is highly parameterized, allowing exploration of numerous internal and environmental factors that may influence a strategy's success. 

## The Rules <a name="rules"></a>

### Rules, In Brief

The ISPT is defined for 3 or more players. Players are matched in pairs and negotiate over how to split a 'pie' of 1 point (see *Initial Setup* below for handling odd number of players). In each pair, one player is randomly chosen to be the **offerer**, the other is the **responder**. The offerer proposes a split (the proportion of the pie that the responder should get) and the other player responds with '**Accept**, **Counter**, or **Reject**. If the responder accepts, then they split the point as proposed. In the next round, the offerer and responder meet again with roles reversed and negotiate over a new pie. If the responder chooses 'Counter', then no points are awarded and both players meet in the next round with offerer and responder roles reversed. Additionally, each player's discount parameter applies, so any points awarded to a player in this round will be reduced by the player's discount factor (the shrinking of the pie). If the responder chooses 'Reject' then the pairing between these two players is eliminated. If a rejection leaves either player unpaired, then their respective discount parameter is applied and in the next round they are randomly paired with a different player. The tournament continues until a specified number of rounds have elapsed or a random termination parameter ends the game.

It is worth noting that players only earn points when they accept an offer or when their offer is accepted. However, if agents are too accepting it opens the door for a hardball player to win the lion's share of points for themselves. Repeated runs of the game explores which offer/response strategies tend to be successful, and how successful they remain after changing environmental conditions.

### Rules, In Depth

#### Initial Setup

Players are randomly matched in pairs. Each match is called a **table**, i.e. a negotiation table. If there are an odd number of players in the tournament then one player is randomly chosen to be tabled with two different players, so that all players have at least one table. At each table is a 'pie' of 1 point to be split between the two players. 

At each table, both players have a **discount factor**, which is their current discount at that table. The discount factors are all initialized to 1.


#### Actions

At each of these initial tables, one player is randomly selected to be the offerer. This player makes an offer between 0 and 1 (inclusive), representing the **proportion** of the pie they are offering to the responder. Specifically, the offerer's available actions are any real value from 0 to 1: $O \in [0, 1]$.

The responder then has 3 available actions: accept, counteroffer, or reject (actions = $\{ Accept, Counter, Reject\}$). At all tables, the offerers simultaneously propose their splits and the responders immediately and simultaneously choose their responses.

Gameplay continues based on the responder's actions as follows:
- **Accepts**: both players split the value of the pie as proposed by the offerer. The points are then multiplied by each player's current discount, then added to their respective scores (all discounts are 1 in the first round but in subsequent rounds discounts may be less). In the next round this table is repeated with the roles reversed. In other words, the same two players are placed at a table with the responder becoming the offerer and the offerer becoming the responder. Their discount factors are again initialized to 1.

- **Counteroffers**: neither player wins any points and each player's discount factor shrinks by their discount parameter. In the next round the players are paired at a table with the roles reversed as in the 'accept' case (offerer becomes responder and vice versa), only now their discount factors have decreased. 

    - Example. Say player A has a discount parameter of 0.8 and a current discount factor of 0.64 at a particular table, playing against B. If the table ends in 'Counter', then A will again play B next round with their offer/responder roles reversed. But A's discount factor becomes $0.64 \cdot 0.8 = 0.512$. Should this table now end in 'Accept', A will get the proportion proposed *times* 0.512.

- **Rejects**: neither player wins any points and the players will not be paired together in the next round. If the responding player would be in zero tables as the result of a rejection, then in the next round they are randomly paired with another player, excluding the offerer. Likewise, if the offerer becomes 'untabled' by rejection then in the next round the offerer is randomly paired with another player, excluding the responder. If a new table is created for the offerer or responder, their discount parameter is applied to their discount at this table. Players randomly selected to participate in new tables have their discount factor initialized to 1. See *Discounts & Tables* below for an in-depth example.

#### Discounts & Tables

Each player's discount parameter determines how rapidly their share of a pie will shrink. At each table, each round that does not end in 'Accept' results in both players' discounts applied to their potential winnings. Since players may be at more than one table simultaneously, each player has a discount factor **per table**, which decreases by their discount parameter each round that does not end in 'Accept'.

If a player comes to a table by being randomly selected (either in the initial round or in later rounds), their discount factor is initialized to 1, otherwise their discount parameter is applied. I.e. players who arrive at a table because in the previous round they rejected or were rejected have their discount parameter applied; players who were randomly selected to participate in the table do not. 

For example, say a tournament includes 3 players, A, B, and C, each with a discount rate of 0.9. A is tabled with B, and A is also tabled with C. Say A is the offerer at both tables, and offers both players 0.25. B rejects the offer but C accepts:

<table>
    <tr>
        <th>Round</th>
        <th>Offerer</th>
        <th>Responder</th>
        <th>Offer</th>
        <th>Response</th>
        <th>Offerer Discount</th>
        <th>Responder Discount</th>
        <th>Points to Offerer</th>
        <th>Points to Responder</th>
    </tr>
    <tr>
        <td>1</td>
        <td>A</td>
        <td>B</td>
        <td>0.25</td>
        <td>Reject</td>
        <td>1</td>
        <td>1</td>
        <td>0</td>
        <td>0</td>
    </tr>
    <tr>
        <td>1</td>
        <td>A</td>
        <td>C</td>
        <td>0.25</td>
        <td>Accept</td>
        <td>1</td>
        <td>1</td>
        <td>0.75</td>
        <td>0.25</td>
    </tr>
</table>

Since C accepted, in the next round A and C will be at a table together with C now offering, A responding, and both discount factors set to 1:
<table>
    <tr>
        <th>Round</th>
        <th>Offerer</th>
        <th>Responder</th>
        <th>Offer</th>
        <th>Response</th>
        <th>Offerer Discount</th>
        <th>Responder Discount</th>
        <th>Points to Offerer</th>
        <th>Points to Responder</th>
    </tr>
    <tr style="background-color: lightgreen;">
        <td>2</td>
        <td>C</td>
        <td>A</td>
        <td> </td>
        <td> </td>
        <td>1</td>
        <td>1</td>
        <td> </td>
        <td> </td>
    </tr>
</table>

Since A and C still have a table in round 2, no new tables need to be generated on their behalf. However, since B rejected, the table between A and B is eliminated, leaving B 'untabled'. So B is randomly paired with another player in the tournament, and the tournament randomly selects player C. B's discount at this table decreases by a factor of 0.9 but C's discount is initialized to 1. The tournament will also randomly choose which player will be the offerer; say it chooses C.

<table>
    <tr>
        <th>Round</th>
        <th>Offerer</th>
        <th>Responder</th>
        <th>Offer</th>
        <th>Response</th>
        <th>Offerer Discount</th>
        <th>Responder Discount</th>
        <th>Points to Offerer</th>
        <th>Points to Responder</th>
    </tr>
    <tr>
        <td>2</td>
        <td>C</td>
        <td>A</td>
        <td> </td>
        <td> </td>
        <td>1</td>
        <td>1</td>
    </tr>
    <tr>
        <td>2</td>
        <td>C</td>
        <td>B</td>
        <td> </td>
        <td> </td>
        <td>1</td>
        <td>0.9</td>
        <td> </td>
        <td> </td>
</table>

If C offers 0.5 to B, and B accepts, then C gets 0.5 points while B gets $0.9 \cdot 0.5 = 0.45$ points:

<table>
    <tr>
        <th>Round</th>
        <th>Offerer</th>
        <th>Responder</th>
        <th>Offer</th>
        <th>Response</th>
        <th>Offerer Discount</th>
        <th>Responder Discount</th>
        <th>Points to Offerer</th>
        <th>Points to Responder</th>
    </tr>
    <tr>
        <td>2</td>
        <td>C</td>
        <td>B</td>
        <td style="background-color: lightgreen;">0.5 </td>
        <td style="background-color: lightgreen;">Accept</td>
        <td>1</td>
        <td>0.9</td>
        <td style="background-color: lightgreen;">0.5</td>
        <td style="background-color: lightgreen;">0.45</td>
</table>


Notice that even though B accepted a 50/50 split, the difference in discounts caused fewer points to be awarded to B.

If B had chosen to 'Counter' instead of 'Accept', then both players play each other in the next round but B's discount becomes 0.81 while C's discount becomes 0.9.

<table>
    <tr>
        <th>Round</th>
        <th>Offerer</th>
        <th>Responder</th>
        <th>Offer</th>
        <th>Response</th>
        <th>Offerer Discount</th>
        <th>Responder Discount</th>
        <th>Points to Offerer</th>
        <th>Points to Responder</th>
    </tr>
    <tr>
        <td>2</td>
        <td>C</td>
        <td>B</td>
        <td>0.5 </td>
        <td>Counter</td>
        <td>1</td>
        <td>0.9</td>
        <td>0</td>
        <td>0</td>
    </tr>
    <tr style="background-color: lightgreen;">
        <td>3</td>
        <td>B</td>
        <td>C</td>
        <td> </td>
        <td></td>
        <td>0.81</td>
        <td>0.9</td>
        <td> </td>
        <td> </td>
    </tr>
</table>


### Game Parameters:
The game currently supports these parameters:
- Length of game: The maximum number of rounds for the tournament. The default length is 1000 rounds. 
- Random termination: allows setting the tournament to randomly terminate after a certain round, with a given probability. For example, random termination can be set to 0.01 at round 1000, meaning the tournament has a 1% chance of terminating each round after round 1000.
- Discount parameters: A discount parameter can be specified for each player, any real number between 0 and 1. Players could all have the same discount parameter or players can individually have different discounts assigned.
- Information Restriction: restricting the information available to players can simulate social networks, in which subsets of players symmetrically have access to each others' histories. Asymmetric topologies can be created as well. By default, the players have complete information. They are aware of all the actions all players have taken in previous rounds (but not the current round). Additionally, they are aware of all discount factors, scores, and game statistics. Currently, the game does not permit responders access to the current round's offers at other tables. Information can be restricted by specifying which players' histories each player has access to. Players have access to their own history regardless. For example, if Player 1's information is restricted to Player 2 and Player 5, then when Player 1 queries the game history they will only receive records of tables where players 1, 2, or 5 were participants. 
- Noise: a noise parameter can be set, any real value in [0, 1], so that any player's response is randomly changed with this probability. If the response is changed, it is changed to one of the two other actions with equal probability. E.g. if the noise parameter is 0.01, and a player responds with 'Accept', then there is a 1% chance the response will be switched. If it is switched, then there is a 50% chance the response will be changed to 'Counter' and a 50% chance the response will be changed to 'Reject'.



### Game Statistics:

Each player of course has a score which is the sum of all points they have earned. Since this number is highly sensitive to the number of tables a player participates in (which itself is random), the game also tracks:
- Average points per round
- Average points per offer

Both these statistics may be meaningful: average points per round perhaps speaks to a player's ability to profit by maintaining relationships, while average points per offer may indicate the player's efficacy as a dealmaker.

## API & Running the game <a name="api"></a>

### Important files

- `agent.py`: defines the `Agent()` class. Subclassing `Agent()` provides a template for constructing new agents and implementing new strategies.
- `agents/` directory: `ispt.py` will look in this directory for agents to instantiate when running the tournament.
- `api.py`: defines classes necessary for running the tournament and exporting results. In particular, the `ISPT()` class defines several methods allowing agents to retrieve game information.
- `constants.py`: various constants used by `api.py` and agents for convenience. E.g. `ACCEPT` is set to 0, the code for the 'Accept' action.
- `data/` directory: default location where the `ISPT()` class will save graphs and csv output.
- `ispt.py`: a script to instatiate agents and the tournament. 

### Running the game

The file `ispt.py` sets up and executes the game. This is broken down into two basic steps:
1. Specify what agents will be playing 
2. Set up the tournament's parameters and play

#### Specifying agents
In `ispt.py`, create the list `player_data`. This should be a list of lists whose elements follow the form: `[agentName, [nameString, otherArguments]`. 

- `agentName`: the filename (without `.py`, and case-insensitive) of the module defining the agent. By default the file `agentName.py` should be in the `agents/` directory. One agent should be defined per file.
- `nameString`: the name string for this agent. This allows you to set a different name for each instance of an agent.
- `otherArguments`: Optional. If your agent's `__init__` function takes additional arguments these can be passed here, separated by commas.

Code exists in `ispt.py` to then instantiate and name all the agents specified in `player_data`, and storing these instances in a list called `players`. It then instatiates the game by calling `ISPT`, described below.
#### Setting parameters & playing

Instantiate the game by calling `ISPT()`:

`game = ISPT(players, discounts=None, default_discount=0.9, info_availability=None, initial_scores=None)`

- `players`: list of agent instances to use as the players in the tournament. 
- `discounts`: list of discount rates to use for each player. If specified, this needs to be the same length as the number of players. The values will be matched element-wise with the `players` list. If not specified, each player gets the default discount rate below.
- `default_discount`: if no `discounts` specified, all players get this discount.
- `info_availability`: dictionary mapping player id's to lists of player id's. Each key is then a player id, and the value it maps to is a list of the player id's the key id player has access to. If this parameter is omitted, the game defaults to complete information: all players can access all past tables.
- `initial_scores`: list of scores for each player. If specified, the game starts at round 1 with these scores assigned. If unspecified, all players start with 0 points.

The `ispt.py` script then stores the game instance in `game`. All that's left is to `play`:

`history = game.play(max_rounds=1000, termination_prob=(100, 0.01), response_noise=0.01, export_csv=False)`

- `max_round`: the absolute maximum number of rounds the tournament will run for.
- `termination_prob`: a 2-item tuple. The first item specifies at which round random termination begins. After this round, the second item specifies the probability the game will terminate at the end of this round.
- `response_noise`: the response noise parameter (see above)
- `export_csv`: if `True`, the game will export the history of actions and scores in `tables.csv` and `stats.csv` to the `data/` directory after the tournament has ended.

The `.play()` method returns the game's history, formatted as a list of `State()` objects, one `State()` for each round. The first item in the list represents the game state at the end of the first round, and so on.

Additionally, graphical output can be generated by the methods:
- `game.graph_scores()`: creates a set of line plots graphing the game statistics by round.
![title](readme_imgs/graphs.png)

- `game.heatmap()`: creates a heatmap showing how players earned points from each other. Each cell displays the points that the 'row' player earned from the 'column' player. E.g. in cell (3, 4) is the amount of points player 3 earned from player 4. In cell (4, 3) is the amount of points player 4 earned from player 3. Since discounts may be different for different players, these two scores aren't necessarily equal.
![title](readme_imgs/heatmap.png)

### Usage

#### Creating agents
The ISPT supports agent duck typing: an agent can be any object that has: 
- a `name` attribute: the default name for the agent (string).
- an `offer(self, table)` method that takes a `TableRecord()` object as a parameter. It returns a number between 0 and 1 representing the split the offerer makes to the responder. E.g. returning `0.1` indicates the offerer offering 10% of the pie to the responder and keeping 90% for themself.
- a `response(self, table, offer)` method that takes a `TableRecord()` object and an `offer` (float between 0 and 1) as parameters. It returns 0 for 'Accept', 1 for 'Counter', or 2 for 'Reject'. Importing all from `constants.py` will import the variables `ACCEPT`, `COUNTER`, and `REJECT`, with the corresponding values already set.

Subclassing the `Agent()` class inherits this structure. 

#### Defining agent strategies
Any logic can be set in an agent's `offer` and `response` methods to determine which action an agent will take. The `table`  parameter will be a `TableRecord()` object representing the table in the game for which the agent must provide an offer or response. I.e. if an agent's `offer` method is called, it means this agent is the offerer at the table passed in, and the agent needs to make an offer to the responder. 

If an agent's `response` method is called, the agent has the current `offer` and `table` passed in available for inspection. While `offer` is simply a number between 0 and 1, the `table` parameter again takes a `TableRecord()` object, which will contain information about the players at this table and current discounts. See **Getting game state & history** below for more information about accessing data in the `TableRecord()` object.


#### Identifying players within the game
Generally, each player is given an id number corresponding to the order in which they are passed in to the tournament, starting at 0. A player's id in the game then corresponds to their index in most lists of game stats. For example, if there are 10 players in a tournament, then at any time the game scores will be a 10-element list, and the 1st player's score is stored at `scores[0]`. The $n$th player's score is at index $n - 1$.

#### Getting game state & history
Various methods have been defined (as `@classmethods`) that allow agents to query the current game state or the game history.

- `ISPT.get_state()`: returns the `State()` object representing the game's current state.

The `State()` object has the following attributes:
- `.tables`: a list of `TableRecord()` objects, representing each table currently existing.
- `.num_players`: number of players in the tournament.
- `.scores`: a list of cumulative scores. The $n$th player's score will be at the $n - 1$ index.
- `.avg_score_per_round`: a list of the cumulative scores divided by the number of rounds.
- `.avg_score_per_offer`: a list of the cumulative scores divided by the number of offers each player has made or received.
- `.table_count`: a list of current number of tables each player is participating in.
- `.cumulative_tables`: a list of cumulative counts of tables per round for each player. I.e. for each round, each table a player participates in contributes +1 to this count.
- `.round`: the current round

A `TableRecord()` object collects the results of a particular table between two players. Note that at the beginning of a round, `.offer` and `.response` will both be `None`. These attributes get populated as the round is played.

- `.players`: a 2-element tuple, the first element is the id of the offerer, the second element is the index of the responder
- `.offerer`: the id of the offerer (available for syntactic convenience)
- `.responder`: the ide of the responder (available for syntactic convenience)
- `.discounts`: a 2-element tuple, the first represents the current discount of the offerer, the second is the current discount of the responder
- `.offer`: The offer that the offerer has made (or `None` if offer has not yet been made).
- `.response`: The responder's response (or `None` if responder hasn't yet responded).

Agents can query the game history using the following:

- `ISPT.get_history(players=[])`: returns the game's history of tables. The history is returned as a list of lists of `TableRecord()` objects, one list for each past round in the tournament. NOTE: if information restriction is used, then this function **automatically** filters out any tables the agent does not have access to see. That is to say, this function returns the full game history available to each player, depending on their information access.
    - `players`: list of player id's for whom the agent requests history. Players not in the list will be filtered out of the return object. If this parameter is unspecified, all available history is returned.

Note: `ISPT.get_history()` is defined to return a list containing one empty list in the first round (`[[]]`). This is so that agents do not all need to make a special case for round 1 when querying history.

#### Examples
For example, this code could go in an agent's `offer` method. It calculates and returns the average offer made out of all offers from players 1, 5, and 9. Note that it is possible they have not yet made any offers, so it includes a default return value of 0.25:

<code>history = ISPT.get_history(players=[1, 5, 9])
offers = []
for round in history:
    offers += [table.offer for table in round]

if offers:
    return sum(offers) / len(offers)
else:
    return 0.25
</code>
    
Another useful call would be `ISPT.get_history()[-1]`, which returns just the previous round (but in the first round it will return an empty list). 

Consider this `response` method that calculates the average offer that was accepted in the last round. If the `offer` passed in is greater than the average offer from the last round, then the agent accepts. Otherwise, it chooses to counter.

<code>last = ISPT.get_history()[-1]
accepted = [table.offer for table in last if table.response == ACCEPT]
if accepted:
    if offer > sum(accepted) / len(accepted):
        return ACCEPT     # ACCEPT=0, COUNTER=1 in constants.py
else:
    return COUNTER
</code>

Note once again that it is possible there were no offers accepted in the previous round, so a default return value is included.

## Future Research <a name="future-research"></a>

### Lines of inquiry

#### Finding dominant strategies

Finding dominant strategies under certain game parameters will be of primary interest. Furthermore, it may be of theoretical and practical interest to determine how much the game parameters can change until a dominant strategy no longer dominates. It may be worth exploring the extent to which a strategy relies on access to information, and the presence or absence of other strategies.

#### As a continuous PD tournament

The iterated shrinking pie game can be viewed as a variation of the iterated PD. In PD, 'cooperation' is binary, while in the shrinking pie, what the offerer considers cooperative may differ from what the responder believes. For example, most people might consider an offer of 0.5 as 'cooperative'. However, is 0.4 still 'cooperation'? What about 0.2? The responder can consider anything on a continuous scale from 0 to 1 as the threshold of cooperation. Furthermore they can modify this belief based on the current game state or historic game states (as can the offerer). The offerer may think they are being quite generous if they offer 0.33, while the responder might find this so low as to reject.

Following this reasoning, translating traditional PD strategies to ISPT strategies may yield interesting results. Do successful IPDT strategies remain successful in the ISPT?

The PD strategy TIT FOR TAT might translate to ISPT as 'offer whatever the last player offered me'. However, what would the response strategy be? TIT FOR TAT would have to choose when a particular offer represented cooperation or defection. Beyond that, the strategy must define how 'defection' in the PD sense should translate to Counter versus Reject.

#### Individual vs. Collective Bargaining

It will be of great interest to see how  sharing information amongst a network affects bargaining efficacy. Do larger networks have an advantage over smaller ones? If so, how large does a network have to be to offset a lower discount parameter in its members?

#### Strategy Families

Can strategies be meaningfully classified into 'families', where a fundamental strategy is implemented across a range of slightly varying agents? This would simplify creation of strategies and exploring successful vs. unsuccessful strategies. Results could indicate if successful strategies are the result of finely tuning to the environment (a few agents within a family are successful) or if it is more important to have robust underlying logic (the family tends to succeed or fail as a whole).

### Additional game parameters
Additional game parameters being developed or considered include:

- Information Delay: information doesn't become instantly available to players across their social networks, but rather takes a certain number of turns to propagate.
- Semi-simultaneous play: Allowing responders access to the current round's offers across all tables, subject to information restrictions. This could permit simulating networks with the ability to compare offers received and coordinate responses.
- Table caps: in the basic version of the game, there is no limit on how many tables a player can participate in simultaneously, and no limit on how many offers they can make or accept. However, we may want to restrict a player to being able to make or accept a certain number of offers per round. This may more closely model negotiations over a finite opportunity. For example, a laborer in the workforce can take on one full time job at a time, so they can only accept one offer per time period and all else being equal, would rationally accept the highest offer. If other players represent firms, then they likewise may have a finite number of job positions to fill. 
- Cost of Doing Business: players must 'pay' a certain number of points each turn in order to stay in the tournament. 
- Elimination: players whose scores drop below some threshold are eliminated from the tournament.
- Variable Pie: rather than every pie be a uniform 1 point, some pies can be larger than others

## References <a name="references"></a>

Rubinstein, Ariel (1982). "Perfect Equilibrium in a Bargaining Model". Econometrica. 50 (1): 97–109. CiteSeerX 10.1.1.295.1434. doi:10.2307/1912531. JSTOR 1912531. 
